# Tutorial for generating reliability field maps and gradient flow vectors:

## Reliability Field maps:  
Field maps show the contribution of the intra (x-axis) and inter (y-axis) variation to the ICC in a two dimensional histogram.

The field maps can be created using any measurable variable, but we focus on edges from a functional connectivity matrix. For a given edge, we plot the intra- and inter-variation on the x- and y-axis respectively revealing ICC (intra- and between-variation) via diagonal lines originating from point 0,0 in a rainbow-like manner. 

## Gradient flow vectors:
Built upon the variability field map, we also utilize the variability gradient flow map (GFM) for assessing the relative impact of reliability optimizations focused on one form of variation versus the other, depending on the current balance of the intra- and inter-individual variations. This allows us to identify how and where the difference in variability between two conditions changes in an optimal or sub-optimal direction towards the highest 

The gradient vector on the field map represents the optimal direction for maximal ICC change on the respective axes of individual variability. For a given ICC change, we normalized its gradient by the contributions of individual variability as compared to the optimal direction


#### Import functions needed for field maps and gradient flow vectors

In [1]:
%matplotlib inline
import numpy as np

## Data used:
 * Functional
     * Every edge from the upper triangle of a 360 x 360 functional connectivity matrix was used as input for the ICC model.
         * N<sub>subs</sub> x edges from upper triangle of functional connectivity matrix
     * The output arrays generated from the ICC model along with our specific data dictionary keys are listed below: 
         * ICC ('icc')
         * raw intra-individual variation ('raww')
         * raw inter-individual variation ('rawb')
         * total variation ('vartotal')
         * mask of failed edges ('totmask')
     * Also included are masked arrays which exclude failed edges. 
         * 'icc_masked', 'raww_masked', 'rawb_masked', 'vartotal_masked', 'totmask_masked'
 * Behavioral
     * N<sub>subs</sub> x 68 behavioral measures “Cognition”, “Emotion”, “Personality” (check “Motor” and “Sensory”)

In [2]:
# Load Data:
data = np.load('../tutorial/example_data/tutorial_data.npy',allow_pickle=True).item()
tasks = [task for task in data.keys()]

print('Task conditions in data variable: %s' % tasks)
print('Dictionary keys within each task condition: %s' % [data_keys for data_keys in data[tasks[0]].keys()])

Task conditions in data variable: ['REST_nogsr', 'REST_gsr']
Dictionary keys within each task condition: ['rawb', 'raww', 'icc', 'vartotal', 'totmask']


#### First obtain descriptives on GSR and no GSR ICC:

In [3]:
for task in data.keys():
    icc = data[task]['icc'][data[task]['totmask']] 
    inter = data[task]['rawb'][data[task]['totmask']] 
    intra = data[task]['raww'][data[task]['totmask']] 
    print('%s ICC mean: %s +/- %s' % (task,np.round(np.mean(icc),3),np.round(np.std(icc),3)))
    print('%s Inter-individual variation mean: %s +/- %s' % (task,np.round(np.mean(inter),3),np.round(np.std(inter),3)))
    print('%s Intra-individual variation mean: %s +/- %s\n ' % (task,np.round(np.mean(intra),3),np.round(np.std(intra),3)))

    

REST_nogsr ICC mean: 0.327 +/- 0.166
REST_nogsr Inter-individual variation mean: 0.006 +/- 0.004
REST_nogsr Intra-individual variation mean: 0.012 +/- 0.004
 
REST_gsr ICC mean: 0.339 +/- 0.175
REST_gsr Inter-individual variation mean: 0.009 +/- 0.007
REST_gsr Intra-individual variation mean: 0.015 +/- 0.005
 


Resting-state scans processed with GSR have greater mean ICC (0.34 +/- 0.18) compared to scans processed without GSR (0.33 +/- 0.17)

In the following, we look at how inter- and intra-individual variation contribute to ICC differences.

## Generating reliability field maps:
 * For this tutorial we compare functional connectivity from data processed with GSR and without GSR.
 * After importing libraries we load in results generated from our relibility model, most importantly ICC, raw intra-individual variation, raw inter-individual variation which we will use for both reliability field maps and gradient flow maps.
 * Light grey lines from the origin, (0,0), mark ICC values starting from 0.1 (shallowest angled bottom line) to 0.9 (steepest most left) - ICC of 0 and 1 are the X and Y axes respectively.
 * We will first create individual field maps with the function **plot_field_map()**.
 * Overlaying the individual field maps with just the contour lines allows visual comparison of inter- and intra-individual variation on ICC (**field_map_overlay()**).
 
 
## 1. Individual fieldmaps for GSR and no GSR
 * _tasks_, _taskcolors_, _taskcmaps_ should include all conditions included in _data_
 * _taskcolors_ are individual colors for each task - can be hex or color name as set here
 * _taskcmaps_ should be color maps corresponding to each _task_ and _taskcolor_

In [4]:
from reliability.reliability_field_maps import plot_field_map

# Set colormaps/colors for field map contours
tasks = ['REST_nogsr',
         'REST_gsr']
taskcolors = {'REST_gsr':'grey',
              'REST_nogsr':'red'}
taskcmaps = {'REST_gsr':'Greys',
             'REST_nogsr':'Reds'}
outpath = False # Not saving any figures for now
lines = True # Show ICC lines
alpha = 1 
thr = 0.0001 # Our threshold for which variability values to include

# Plot individual field maps:
for num,task in enumerate(['REST_nogsr','REST_gsr']):
    x = data[task]['raww'][data[task]['totmask']]
    y = data[task]['rawb'][data[task]['totmask']]
    xx1,yy1,normalized,fig = plot_field_map(x,y,taskcolors[task],taskcmaps[task],alpha,lines,outpath,thr=0.0001,gridsize=100,
                         overlay=False,cbar_option=True,figSize=(12,10),xyLim=95,shade=True,addContourLines=True,plotstyle=['nokde','scatter'],bins=500)



FileNotFoundError: [Errno 2] File b'/anaconda2/envs/py36/lib/python3.6/site-packages/reliability/misc/Yeo7_to_Glasser360_labels/181Yeo7matchlh.csv' does not exist: b'/anaconda2/envs/py36/lib/python3.6/site-packages/reliability/misc/Yeo7_to_Glasser360_labels/181Yeo7matchlh.csv'

## 2. Field map overlap for GSR and no GSR

In [ ]:
from reliability.reliability_field_maps import plot_field_map_overlay

# Not saving any figures for now.
outpath = False
lines = True
alpha = 1

# Choosing conditions to compare:
taskcombos = [['REST_nogsr',
               'REST_gsr']]

plot_field_map_overlay(taskcombos,data,taskcolors,taskcmaps,alpha,lines,outpath,
                      cbar_option=True,figSize=(13,10),xlim=(0,0.025),ylim=(0,0.025),
                     shade=False,thr=0.0001)

## Generating gradient flow vector angular histograms:
 * Gradient flow vectors are useful understand optimality of change in variation with respect to improved reliability. Gradient flow angular histograms show the frequency of edges for each angle.
 * Gradient flow histograms are created using **gradient_flow_histogram()**.
     * 1. intra- and inter-individual variation is used to calculate standardized gradient flow vectors and the count for each angle is plotted.
     * 2. Each angle is counted and the total number is plotted according to the gradient flow circular colormap.

In [ ]:
from reliability.gradient_flow_vectors import calc_icc_vectors,pah
from reliability.variability_utils import gradientFlowCmaps

##########################################
# Turn calc_icc_vectors input as pd.df
# Headers with edge names for 360

# Paired masking:
# Input data should be masked such that each intra- and inter-individual variation value are paired between the two conditions
bothmask = data[tasks[0]]['totmask'] * data[tasks[1]]['totmask']

parcdf = {}
measures = ['icc','raww','rawb']
for task in tasks:
    parcdf[task] = {}
    for m in measures:
        parcdf[task][m] = data[task][m][bothmask]

# Calculate vector parameters:
df = calc_icc_vectors(parcdf[tasks[0]]['raww'],parcdf[tasks[0]]['rawb'],parcdf[tasks[0]]['icc'],
                      parcdf[tasks[1]]['raww'],parcdf[tasks[1]]['rawb'],parcdf[tasks[1]]['icc'],
                      tasks[0],tasks[1])

# Gradient flow histogram plot options:
bin_threshold = 5 # degrees covered per histogram bar
outpath = False # No saving for now.
title = '%s - %s' % (tasks[1],tasks[0])

# Plot gradient flow histogram:
pah(df['theta0'],bin_threshold,gradientFlowCmaps['complete'],title,outpath)

